In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam

model = Sequential()
model.add(Conv2D(16, (5, 5), padding='same', input_shape=(48, 48, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (5, 5), padding='same', ))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (5, 5), padding='same', ))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation="softmax"))
# model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 16)        1216      
_________________________________________________________________
activation (Activation)      (None, 48, 48, 16)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        12832     
_________________________________________________________________
activation_1 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 32)        2

In [ ]:
batch_size = 16

from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                   # rotation_range=10,
                                   # width_shift_range=0.2,
                                   # height_shift_range=0.2,
                                   # horizontal_flip=True,
                                   validation_split=0.2)

val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = train_datagen.flow_from_directory('./Datawhale_人脸情绪识别_数据集/train/', 
                                               color_mode='rgb',
                                               batch_size=batch_size, 
                                               class_mode='categorical',
                                               target_size=(48, 48),
                                               shuffle=True,
                                               subset = 'training') 

val_data = val_datagen.flow_from_directory('./Datawhale_人脸情绪识别_数据集/train/', 
                                           color_mode='rgb',
                                           batch_size=batch_size, 
                                           class_mode='categorical',
                                           target_size=(48, 48),
                                           shuffle=False,
                                           subset = 'validation')

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: './Datawhale_人脸情绪识别_数据集/train/'

In [ ]:
model.fit_generator(
        train_data,
        epochs=5,
        validation_data=val_data,
        validation_steps=200)
model.save_weights('first_try.h5') 

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        './Datawhale_人脸情绪识别_数据集/',
        target_size=(48, 48),
        color_mode="rgb",
        shuffle = False,
        class_mode='categorical',
        batch_size=10)

filenames = test_generator.filenames
nb_samples = len(filenames)
predict = model.predict_generator(test_generator,steps = nb_samples)

In [ ]:
train_data.class_indices

In [ ]:
import numpy as np
import pandas as pd
cls_name = np.array(['angry', 'disgusted', 'fearful', 'happy','neutral', 'sad', 'surprised'])

In [ ]:
submit_df = pd.DataFrame({'name': filenames, 'label': cls_name[predict.argmax(1)]})

In [ ]:
submit_df.head()

In [ ]:
submit_df = submit_df[submit_df['name'].apply(lambda x: 'test' in x )]
submit_df['name'] = submit_df['name'].apply(lambda x: x.split('/')[-1])

In [ ]:
submit_df = submit_df.sort_values(by='name')
submit_df.to_csv('submit.csv', index=None)